# Template Definition

In [1]:
ROOT_FOLDER = "/Users/marcocalamo/KUL/Dataset/"

In [2]:
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
prompt_template = PromptTemplate.from_template(
"""
You will be asked by the user to create a plant UMl model from specification text. Do so in the most
clear way possible, avoid class properties and assign molteplicity. 

Do not include attributes for classes. For example the class Book would be:

class Book{{}}

Use only bi-directional arc for relations and no description. For example a relation between
the class Book and the class Page, if the Book can have from one to many pages and the 
pages could have exactly one book, would be:

Book "1..1" -- "1..*" Page

Adapt the cardinality to each case. If the cardinality would be "0..*", the default one, omit it.

The plantuml has to be the class diagram. In generating the diagram perform this steps in order

1. Extract class from text
2. Extract relations form text
3. Assign the relation to the corresponding class
4. Add cardinality to the relations

Put everything in this order: first all classes and then all relations. In our example would be:

@startuml

class Book{{}}
class Page{{}}

Book "1..1" -- "1..*" Page

@enduml

Output plantuml without futher text or explaination.

##############

Here is an example of how should be done. 

The specificartion example text is:

Alpha Insurance is an insurance company that provides its clients with various types of insurance policies. 
As soon as a customer addresses Alpha Insurance, a broker is assigned to follow the customer’s file. 
The broker is registered in the system, so that when a customer calls, based on the contract, 
the help desk can immediately trace who is the customer's first account manager.
After the broker is assigned to the customer, the latter indicates which type(s) of insurance policy they would like to sign for,
so the broker could, depending on the case, either assess the customer’s profile on the spot or send the customer’s file
for analysis to the head office. After the customer’s profile has been assessed and the customer has been deemed trustworthy,
a preliminary contract/offer on an insurance product is made to the customer either in person or by email.
(Such offers can also be extended to already existing customers.) 
If the customer agrees to the offer, the contract is signed by both parties.
After the signing of the contract, the client enjoys the coverage and is invoiced
(monthly or yearly – depending on the choice made in the contract) according to the price of the insurance product they bought.

In case the insured event happens, a customer should send a claim for compensation.
Then the company opens one or several claim cases (e.g. in case of an accident,
often material damage & physical damage are handled separately). Once the case file is complete, 
it is sent for assessment by different estimators based on their area of expertise.
According to the reports issued by the estimators, it is decided whether the claim case is approved. 
In case of approval the compensation decision is registered that stipulates which costs are eligible for (partial) refund.
For the supplied documents, the sum of compensation is calculated and the compensation is paid to the customer’s account. 

The estimators’ reports must be stored in the database for at least one year after the payment of compensation for legal purposes. 

The corresponding uml is:

@startuml

class Customer {{}}
class InsurancePolicy {{}}
class Contract {{}}
class Invoice {{}}
class BrokerCustomerAssignment {{}}
class Broker {{}}
class ClaimCase {{}}
class Report {{}}
class Estimator {{}}
class CompensationPayment {{}}

Contract "0..*" -- "1" Customer
Contract "1" -- "0..*" Invoice
Contract "0..*" -- "1" Airline
Contract "1" -- "0..*" InsurancePolicy
Contract "1" -- "0..*" ClaimCase

ClaimCase "1" -- "0..*" CompensationPayment
ClaimCase "1" -- "0..*" Report
Report "0..*" -- "1" Estimator
BrokerCustomerAssignment "0..1" -- "1" Customer
Broker "1" -- "0..*" BrokerCustomerAssignment

@enduml

Here is another example: 

Specification text: 

An oil company possesses a number of gas stations spread across the country. Customers can pay their refuel turn in
cash at the counter or with a personal credit card, but they can also ask for a refuel card.  
When using their refuel card, customers do not pay the refuel turn immediately.
Rather, at the end of each month, the holder of the fuel card receives a monthly invoice with the amounts of
all the refuel turns paid with the fuel card.  On the monthly invoice discounts are given depending on the turnover 
of the customer during the invoiced month.  Invoicing and paying occurs in any order:
one can already receive a second invoice before having paid the first one. 
But obviously, each invoice must first have been sent before a payment can be received.
Each refuel turn only appears on the invoice of the month of the refuel turn. If a customer doesn't pay the invoice, 
the unpaid refuel turns are not added to the next month’s invoice. Instead, the invoice is simply sent again to the customer to
remind him/her of the payment due. If a customer repeatedly does not pay the invoices (e.g. two months in a row),
then the customer is suspended and his/her card is blocked until the bills have been paid.  
The exact threshold to block a customer is not 100% fixed: very good customers are given a bit more slack than new customers. 
If the bills are settled, a customer may return to the normal state.
Each gas station has a number of pumps that each contain a particular type of gasoline. 
Obviously only one customer can refuel his/her car at a time at a given pump. When the nozzle is taken,
the refueling can start. When the nozzle is put back the refueling is stopped. Between those two moment, 
no other customer can use this pump. As soon as the nozzle is put back, the pump becomes available for the next customer.
Each pump has a separate reservoir that must be refilled when the quantity of gasoline drops below a certain preset reordering level.  
Refilling can occur at any time.  
The system should produce a daily report indicating which pump needs to be refilled and for each pump the current level of 
gasoline and the quantity to order. As the bottom of a tank can contain dirt, when the fuel drops below a critical level, 
the pump is automatically blocked so that no refueling can occur with this pump. For the same reason, 
when refilling a tank that was still above the critical level, the pump is temporarily blocked by the gas station responsible for 
approximatively an hour, so that dirt can sink to the bottom. The pump is released when the gas station responsible deems that 
refueling with this pump is safe again.

The plant uml is:

@startuml

class CardHolder
class Invoice {{}}
class InvoiceLine{{}}
class GasStation{{}}
class Pump {{}}
class RefuelTurn {{}}
class CashTurn {{}}

Pump "1" -- "0..*" RefuelTurn
Pump "1" -- "0..*" CashTurn
Pump "0..*" -- "1" GasStation
CardHolder "1" -- "0..*" Invoice
CardHolder "1" -- "0..*" RefuelTurn
Invoice "1" -- "0..*" InvoiceLine
InvoiceLine "0..1" -- "1" RefuelTurn

@enduml

##############

The specification text is:

{text}

##############

Based on the example, the uml output is:
"""
)

In [3]:
import sys
import threading
from time import sleep
try:
    import thread
except ImportError:
    import _thread as thread

def quit_function(fn_name):
    # print to stderr, unbuffered in Python 2.
    print('{0} took too long'.format(fn_name), file=sys.stderr)
    sys.stderr.flush() # Python 3 stderr is likely buffered.
    thread.interrupt_main() # raises KeyboardInterrupt
    
def exit_after(s):
    '''
    use as decorator to exit process if 
    function takes longer than s seconds
    '''
    def outer(fn):
        def inner(*args, **kwargs):
            timer = threading.Timer(s, quit_function, args=[fn.__name__])
            timer.start()
            try:
                result = fn(*args, **kwargs)
            finally:
                timer.cancel()
            return result
        return inner
    return outer

In [4]:
import os
from tqdm import tqdm
from docx import Document

from langchain_core.tracers.context import tracing_v2_enabled

@exit_after(360)
def run_chain(chain, text):
    return chain.invoke({"text": text})

def process_subfolders_with_chain(root_folder_path, chain, type=''):
    """
    Explores subfolders of the root folder (depth 1), processes each subfolder's `text.txt`
    with the provided LangChain chain, and saves the result in a new file in the same folder.

    Args:
        root_folder_path (str): Path to the root folder.
        chain: A LangChain chain instance to process text inputs.
    """
    for subfolder_name in tqdm(os.listdir(root_folder_path)):
        subfolder_path = os.path.join(root_folder_path, subfolder_name)
        
        # Ensure the current item is a subfolder
        if os.path.isdir(subfolder_path):
            text_file_path = os.path.join(subfolder_path, "text.txt")
            
            # Check if `text.txt` exists in the subfolder
            if not os.path.isfile(text_file_path):
                # If `text.txt` is missing, check for any .docx file in the subfolder
                docx_files = [f for f in os.listdir(subfolder_path) if f.endswith(".docx")]
                if docx_files:
                    docx_file_path = os.path.join(subfolder_path, docx_files[0])
                    # Extract content from the .docx file
                    doc = Document(docx_file_path)
                    text_content = "\n".join([paragraph.text for paragraph in doc.paragraphs])

                    # Save the extracted content to `text.txt`
                    with open(text_file_path, "w", encoding="utf-8") as text_file:
                        text_file.write(text_content)

            # Recheck if `text.txt` now exists
            if os.path.isfile(text_file_path):
                with open(text_file_path, "r", encoding="utf-8") as file:
                    text = file.read()

                with tracing_v2_enabled():
                    # Call the LangChain chain with the input dictionary
                    try:
                        result = run_chain(chain, text)
                    except:
                        reuslt = ""

                # Save the result to a new file in the same subfolder
                result_file_path = os.path.join(subfolder_path, f"result_few2_{type}.txt")
                with open(result_file_path, "w", encoding="utf-8") as result_file:
                    result_file.write(result)

In [5]:
def delete_result_txt_files(root_folder_path):
    """
    Deletes every .txt file that starts with 'result_' in the subfolders of the root folder (depth 1).

    Args:
        root_folder_path (str): Path to the root folder.
    """
    for subfolder_name in os.listdir(root_folder_path):
        subfolder_path = os.path.join(root_folder_path, subfolder_name)
        
        # Ensure the current item is a subfolder
        if os.path.isdir(subfolder_path):
            for file_name in os.listdir(subfolder_path):
                if file_name.startswith("result_") and file_name.endswith(".txt"):
                    file_path = os.path.join(subfolder_path, file_name)
                    os.remove(file_path)

In [6]:
#delete_result_txt_files(ROOT_FOLDER)

# Two Shot Open-AI

In [6]:
from dotenv import load_dotenv
assert load_dotenv()

In [7]:
MODEL_OPEN_AI = ["o3-mini", "gpt-4o-mini", "gpt-4o", "gpt-4-turbo", "gpt-3.5-turbo"]

In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from IPython.display import display_markdown

In [9]:
def open_ai_two(model):
    model_ = ChatOpenAI(model=model)
    chain = prompt_template | model_ | StrOutputParser()
    process_subfolders_with_chain(ROOT_FOLDER, chain, type=model)

In [12]:
def open_ai_make_example(model):
    model = ChatOpenAI(model=model)
    chain = prompt_template | model | StrOutputParser()
    res = chain.invoke({"text": """Alpha Insurance is an insurance company that provides its clients with various types of insurance policies. 
As soon as a customer addresses Alpha Insurance, a broker is assigned to follow the customer’s file. The broker is registered in the system, so that when a customer calls, based on the contract, the help desk can immediately trace who is the customer's first account manager. After the broker is assigned to the customer, the latter indicates which type(s) of insurance policy they would like to sign for, so the broker could, depending on the case, either assess the customer’s profile on the spot or send the customer’s file for analysis to the head office. After the customer’s profile has been assessed and the customer has been deemed trustworthy , a preliminary contract/offer on an insurance product is made to the customer either in person or by email. (Such offers can also be extended to already existing customers.) If the customer agrees to the offer, the contract is signed by both parties. After the signing of the contract, the client enjoys the coverage and is invoiced (monthly or yearly – depending on the choice made in the contract) according to the price of the insurance product they bought.

In case the insured event happens, a customer should send a claim for compensation. Then the company opens one or several claim cases (e.g. in case of an accident, often material damage & physical damage are handled separately). Once the case file is complete, it is sent for assessment by different estimators based on their area of expertise. According to the reports issued by the estimators, it is decided whether the claim case is approved. In case of approval the compensation decision is registered that stipulates which costs are eligible for (partial) refund.  For the supplied documents, the sum of compensation is calculated and the compensation is paid to the customer’s account.  The estimators’ reports must be stored in the database for at least one year after the payment of compensation for legal purposes. 

"""})
    return res

In [13]:
display_markdown(open_ai_make_example(MODEL_OPEN_AI[0]), raw=True)

@startuml

class Customer {}
class InsurancePolicy {}
class Contract {}
class Invoice {}
class BrokerCustomerAssignment {}
class Broker {}
class ClaimCase {}
class Report {}
class Estimator {}
class CompensationPayment {}

Contract "0..*" -- "1" Customer
Contract "1" -- "0..*" Invoice
Contract "0..*" -- "1" Airline
Contract "1" -- "0..*" InsurancePolicy
Contract "1" -- "0..*" ClaimCase

ClaimCase "1" -- "0..*" CompensationPayment
ClaimCase "1" -- "0..*" Report
Report "0..*" -- "1" Estimator
BrokerCustomerAssignment "0..1" -- "1" Customer
Broker "1" -- "0..*" BrokerCustomerAssignment

@enduml

In [14]:
for model in MODEL_OPEN_AI[0:1]:
    print(f"Few shot with {model}")
    open_ai_two(model)

Few shot with o3-mini


100%|███████████████████████████████████████████| 48/48 [14:46<00:00, 18.46s/it]


## Two Shot Anthropic

In [9]:
from langchain_anthropic import ChatAnthropic

In [10]:
MODEL_ANTHROPIC = ["claude-3-7-sonnet-20250219"]

In [11]:
def anthropic_make_example(model):
    model = ChatAnthropic(model=model,temperature=0,
    max_tokens=1024,
    timeout=None,
    max_retries=2,)
    chain = prompt_template | model | StrOutputParser()
    res = chain.invoke({"text": """Alpha Insurance is an insurance company that provides its clients with various types of insurance policies. 
As soon as a customer addresses Alpha Insurance, a broker is assigned to follow the customer’s file. The broker is registered in the system, so that when a customer calls, based on the contract, the help desk can immediately trace who is the customer's first account manager. After the broker is assigned to the customer, the latter indicates which type(s) of insurance policy they would like to sign for, so the broker could, depending on the case, either assess the customer’s profile on the spot or send the customer’s file for analysis to the head office. After the customer’s profile has been assessed and the customer has been deemed trustworthy , a preliminary contract/offer on an insurance product is made to the customer either in person or by email. (Such offers can also be extended to already existing customers.) If the customer agrees to the offer, the contract is signed by both parties. After the signing of the contract, the client enjoys the coverage and is invoiced (monthly or yearly – depending on the choice made in the contract) according to the price of the insurance product they bought.

In case the insured event happens, a customer should send a claim for compensation. Then the company opens one or several claim cases (e.g. in case of an accident, often material damage & physical damage are handled separately). Once the case file is complete, it is sent for assessment by different estimators based on their area of expertise. According to the reports issued by the estimators, it is decided whether the claim case is approved. In case of approval the compensation decision is registered that stipulates which costs are eligible for (partial) refund.  For the supplied documents, the sum of compensation is calculated and the compensation is paid to the customer’s account.  The estimators’ reports must be stored in the database for at least one year after the payment of compensation for legal purposes. 

"""})
    return res

In [12]:
def anthropic_two(model):
    model_ = ChatAnthropic(model=model)
    chain = prompt_template | model_ | StrOutputParser()
    process_subfolders_with_chain(ROOT_FOLDER, chain, type=model)

In [13]:
display_markdown(anthropic_make_example(MODEL_ANTHROPIC[0]), raw=True)

@startuml

class Customer {}
class Broker {}
class InsurancePolicy {}
class Contract {}
class Invoice {}
class Claim {}
class ClaimCase {}
class Estimator {}
class Report {}
class CompensationDecision {}
class CompensationPayment {}

Customer "1" -- "0..*" Contract
Customer "1" -- "0..*" Claim
Broker "1" -- "0..*" Customer
Contract "1" -- "0..*" InsurancePolicy
Contract "1" -- "0..*" Invoice
Claim "1" -- "1..*" ClaimCase
ClaimCase "0..*" -- "1..*" Estimator
Estimator "1" -- "0..*" Report
ClaimCase "1" -- "0..*" Report
ClaimCase "1" -- "0..1" CompensationDecision
CompensationDecision "1" -- "0..1" CompensationPayment

@enduml

In [14]:
for model in MODEL_ANTHROPIC:
    print(f"Two shot with {model}")
    anthropic_two(model)

Two shot with claude-3-7-sonnet-20250219


100%|███████████████████████████████████████████| 48/48 [03:21<00:00,  4.21s/it]


# Two Shot Open LLM

In [16]:
import torch, gc

## Deepseek

In [15]:
from langchain_deepseek import ChatDeepSeek

In [16]:
MODEL_DEEPSEEK = ["deepseek-chat"] #, "deepseek-reasoner"]

In [17]:
def deepseek_make_example(model):
    model = llm = ChatDeepSeek(
            model=model,
            temperature=0,
            max_tokens=None,
            timeout=None,
            max_retries=2,
            # api_key="...",
            # other params...
        )
    chain = prompt_template | model | StrOutputParser()
    res = chain.invoke({"text": """Alpha Insurance is an insurance company that provides its clients with various types of insurance policies. 
As soon as a customer addresses Alpha Insurance, a broker is assigned to follow the customer’s file. The broker is registered in the system, so that when a customer calls, based on the contract, the help desk can immediately trace who is the customer's first account manager. After the broker is assigned to the customer, the latter indicates which type(s) of insurance policy they would like to sign for, so the broker could, depending on the case, either assess the customer’s profile on the spot or send the customer’s file for analysis to the head office. After the customer’s profile has been assessed and the customer has been deemed trustworthy , a preliminary contract/offer on an insurance product is made to the customer either in person or by email. (Such offers can also be extended to already existing customers.) If the customer agrees to the offer, the contract is signed by both parties. After the signing of the contract, the client enjoys the coverage and is invoiced (monthly or yearly – depending on the choice made in the contract) according to the price of the insurance product they bought.

In case the insured event happens, a customer should send a claim for compensation. Then the company opens one or several claim cases (e.g. in case of an accident, often material damage & physical damage are handled separately). Once the case file is complete, it is sent for assessment by different estimators based on their area of expertise. According to the reports issued by the estimators, it is decided whether the claim case is approved. In case of approval the compensation decision is registered that stipulates which costs are eligible for (partial) refund.  For the supplied documents, the sum of compensation is calculated and the compensation is paid to the customer’s account.  The estimators’ reports must be stored in the database for at least one year after the payment of compensation for legal purposes. 

"""})
    return res

In [18]:
def deepseek_two(model):
    model_ = ChatDeepSeek(model=model)
    chain = prompt_template | model_ | StrOutputParser()
    process_subfolders_with_chain(ROOT_FOLDER, chain, type=model)

In [19]:
%%time
display_markdown(deepseek_make_example(MODEL_DEEPSEEK[0]), raw=True)

@startuml

class Customer {}
class InsurancePolicy {}
class Contract {}
class Invoice {}
class BrokerCustomerAssignment {}
class Broker {}
class ClaimCase {}
class Report {}
class Estimator {}
class CompensationPayment {}

Contract "0..*" -- "1" Customer
Contract "1" -- "0..*" Invoice
Contract "1" -- "0..*" InsurancePolicy
Contract "1" -- "0..*" ClaimCase

ClaimCase "1" -- "0..*" CompensationPayment
ClaimCase "1" -- "0..*" Report
Report "0..*" -- "1" Estimator
BrokerCustomerAssignment "0..1" -- "1" Customer
Broker "1" -- "0..*" BrokerCustomerAssignment

@enduml

CPU times: user 101 ms, sys: 10.2 ms, total: 111 ms
Wall time: 12.9 s


In [20]:
for model in MODEL_DEEPSEEK:
    print(f"Two shot with {model}")
    deepseek_two(model)

Two shot with deepseek-chat


100%|███████████████████████████████████████████| 48/48 [06:20<00:00,  7.92s/it]


## Ollama

In [17]:
from langchain_ollama import ChatOllama

In [18]:
MODEL_OLLAMA = [] #["llama3.2:3b-text-fp16"]

In [19]:
def ollama_two(model):
    model_ = ChatOllama(
        model=model,
        temperature=0,
        timeout = 3,
    )
    chain = prompt_template | model_ | StrOutputParser()
    process_subfolders_with_chain(ROOT_FOLDER, chain, type=model)

In [20]:
def ollama_make_example(model):
    model = ChatOllama(
        model=model,
        temperature=0,
        timeout = 3,
    )
    chain = prompt_template | model | StrOutputParser()
    res = chain.invoke({"text": """Alpha Insurance is an insurance company that provides its clients with various types of insurance policies. 
As soon as a customer addresses Alpha Insurance, a broker is assigned to follow the customer’s file. The broker is registered in the system, so that when a customer calls, based on the contract, the help desk can immediately trace who is the customer's first account manager. After the broker is assigned to the customer, the latter indicates which type(s) of insurance policy they would like to sign for, so the broker could, depending on the case, either assess the customer’s profile on the spot or send the customer’s file for analysis to the head office. After the customer’s profile has been assessed and the customer has been deemed trustworthy , a preliminary contract/offer on an insurance product is made to the customer either in person or by email. (Such offers can also be extended to already existing customers.) If the customer agrees to the offer, the contract is signed by both parties. After the signing of the contract, the client enjoys the coverage and is invoiced (monthly or yearly – depending on the choice made in the contract) according to the price of the insurance product they bought.

In case the insured event happens, a customer should send a claim for compensation. Then the company opens one or several claim cases (e.g. in case of an accident, often material damage & physical damage are handled separately). Once the case file is complete, it is sent for assessment by different estimators based on their area of expertise. According to the reports issued by the estimators, it is decided whether the claim case is approved. In case of approval the compensation decision is registered that stipulates which costs are eligible for (partial) refund.  For the supplied documents, the sum of compensation is calculated and the compensation is paid to the customer’s account.  The estimators’ reports must be stored in the database for at least one year after the payment of compensation for legal purposes. 

"""})
    return res
    

In [21]:
#display_markdown(ollama_make_example(MODEL_OLLAMA[0]), raw=True)

In [22]:
for model in MODEL_OLLAMA:
    print(f"Few shot with {model}")
    ollama_zero(model)
    gc.collect()
    torch.mps.empty_cache()

## Huggingface

In [23]:
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace, HuggingFaceEndpoint
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [24]:
MODEL_HUGGINGFACE = [] #["Qwen/Qwen2.5-3B-Instruct", "microsoft/Phi-3-mini-4k-instruct", "google/gemma-2-27b-it"]

In [25]:
def huggingface_two(model):
    model_id = model
    
    llm = HuggingFaceEndpoint(
        repo_id=model_id,
        task="text-generation",
        max_new_tokens=2048,
        do_sample=False,
        repetition_penalty=1.03,
        temperature=0.01,
    )

    chat = ChatHuggingFace(llm=llm, verbose=True)
    
    chain = prompt_template | chat | StrOutputParser()
    process_subfolders_with_chain(ROOT_FOLDER, chain, type=model_id.replace("/","_"))

In [26]:
def huggingface_make_example(model):

    model_id = model
    
    

    llm = HuggingFacePipeline.from_model_id(
        model_id=model_id,
        task="text-generation",
        pipeline_kwargs={"temperature": 0.1, "max_new_tokens": 1024}
    )

    chat = ChatHuggingFace(llm=llm, verbose=True)
    
    chain = prompt_template | chat | StrOutputParser()
    res = chain.invoke({"text": """Alpha Insurance is an insurance company that provides its clients with various types of insurance policies. 
        As soon as a customer addresses Alpha Insurance, a broker is assigned to follow the customer’s file. The broker is registered in the system, so that when a customer calls, based on the contract, the help desk can immediately trace who is the customer's first account manager. After the broker is assigned to the customer, the latter indicates which type(s) of insurance policy they would like to sign for, so the broker could, depending on the case, either assess the customer’s profile on the spot or send the customer’s file for analysis to the head office. After the customer’s profile has been assessed and the customer has been deemed trustworthy , a preliminary contract/offer on an insurance product is made to the customer either in person or by email. (Such offers can also be extended to already existing customers.) If the customer agrees to the offer, the contract is signed by both parties. After the signing of the contract, the client enjoys the coverage and is invoiced (monthly or yearly – depending on the choice made in the contract) according to the price of the insurance product they bought.
        
        In case the insured event happens, a customer should send a claim for compensation. Then the company opens one or several claim cases (e.g. in case of an accident, often material damage & physical damage are handled separately). Once the case file is complete, it is sent for assessment by different estimators based on their area of expertise. According to the reports issued by the estimators, it is decided whether the claim case is approved. In case of approval the compensation decision is registered that stipulates which costs are eligible for (partial) refund.  For the supplied documents, the sum of compensation is calculated and the compensation is paid to the customer’s account.  The estimators’ reports must be stored in the database for at least one year after the payment of compensation for legal purposes. 
        
        """})
    return res


In [27]:
#display_markdown(huggingface_make_example(MODEL_HUGGINGFACE[0]), raw=True)

In [28]:
for model in MODEL_HUGGINGFACE:
    print(f"Few shot with {model}")
    huggingface_two(model)
    gc.collect()
    torch.mps.empty_cache()

## Mlx-LLM

In [29]:
from langchain_community.llms import MLXPipeline
from langchain_community.chat_models import ChatMLX

In [54]:
MODEL_MLX = ["mlx-community/phi-4-8bit",
             "mlx-community/Falcon3-10B-Instruct-8bit", 
             "mlx-community/Qwen2.5-14B-Instruct-4bit",
             "mlx-community/Mistral-7B-Instruct-v0.3-4bit",
             "mlx-community/DeepSeek-R1-Distill-Qwen-7B-8bit",
             "mlx-community/Llama-3.2-3B-Instruct",
             "mlx-community/gemma-2-9b-8bit",
             #"mlx-community/gemma-2-27b-it-4bit",
            # "mlx-community/Mamba-Codestral-7B-v0.1-8bit",
            #"mlx-community/CodeLlama-13b-Instruct-hf-4bit-MLX"
            ]

In [31]:
def mlx_two(model):
    llm = MLXPipeline.from_model_id(
        model_id=model,
        pipeline_kwargs={"max_tokens": 15_000, "temp": 0.1},
    )
    chat = ChatMLX(llm=llm)
    chain = prompt_template | chat | StrOutputParser()
    process_subfolders_with_chain(ROOT_FOLDER, chain, type=model.replace("/","_"))

In [32]:
def mlx_make_example(model):
    llm = MLXPipeline.from_model_id(
        model_id=model,
        pipeline_kwargs={"max_tokens": 2000, "temp": 0.7},
    )
    chat = ChatMLX(llm=llm)
    chain = prompt_template | chat | StrOutputParser()
    res = chain.invoke({"text": """Alpha Insurance is an insurance company that provides its clients with various types of insurance policies. 
As soon as a customer addresses Alpha Insurance, a broker is assigned to follow the customer’s file. The broker is registered in the system, so that when a customer calls, based on the contract, the help desk can immediately trace who is the customer's first account manager. After the broker is assigned to the customer, the latter indicates which type(s) of insurance policy they would like to sign for, so the broker could, depending on the case, either assess the customer’s profile on the spot or send the customer’s file for analysis to the head office. After the customer’s profile has been assessed and the customer has been deemed trustworthy , a preliminary contract/offer on an insurance product is made to the customer either in person or by email. (Such offers can also be extended to already existing customers.) If the customer agrees to the offer, the contract is signed by both parties. After the signing of the contract, the client enjoys the coverage and is invoiced (monthly or yearly – depending on the choice made in the contract) according to the price of the insurance product they bought.

In case the insured event happens, a customer should send a claim for compensation. Then the company opens one or several claim cases (e.g. in case of an accident, often material damage & physical damage are handled separately). Once the case file is complete, it is sent for assessment by different estimators based on their area of expertise. According to the reports issued by the estimators, it is decided whether the claim case is approved. In case of approval the compensation decision is registered that stipulates which costs are eligible for (partial) refund.  For the supplied documents, the sum of compensation is calculated and the compensation is paid to the customer’s account.  The estimators’ reports must be stored in the database for at least one year after the payment of compensation for legal purposes. 

"""})
    return res


In [33]:
display_markdown(mlx_make_example(MODEL_MLX[2]), raw=True)

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

@startuml

class Customer {}
class InsurancePolicy {}
class Contract {}
class Invoice {}
class BrokerCustomerAssignment {}
class Broker {}
class ClaimCase {}
class Report {}
class Estimator {}
class CompensationPayment {}

Contract "0..*" -- "1" Customer
Contract "1" -- "0..*" Invoice
Contract "0..*" -- "1" InsurancePolicy
Contract "1" -- "0..*" ClaimCase

ClaimCase "1" -- "0..*" CompensationPayment
ClaimCase "1" -- "0..*" Report
Report "0..*" -- "1" Estimator
BrokerCustomerAssignment "0..1" -- "1" Customer
Broker "1" -- "0..*" BrokerCustomerAssignment

@enduml

In [34]:
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

In [55]:
for model in MODEL_MLX[5:]:
    print(f"Few shot with {model}")
    mlx_two(model)
    import gc, torch
    gc.collect()
    torch.mps.empty_cache()

Few shot with mlx-community/Llama-3.2-3B-Instruct


Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

run_chain took too long                          | 3/44 [00:12<02:35,  3.80s/it]
run_chain took too long                         | 13/44 [06:49<08:03, 15.59s/it]
100%|███████████████████████████████████████████| 44/44 [16:04<00:00, 21.91s/it]


### Clean Cache

In [36]:
import gc, torch
gc.collect()
torch.mps.empty_cache()